In [ ]:
import os
import sys
import subprocess

import numpy as np
import pandas as pd

from io import StringIO

os.getcwd()

'/home/yotamfr/development/skempi/src'

In [2]:
skempi_df = pd.read_excel(os.path.join('../data', 'SKEMPI_1.1.xlsx'))
skempi_df.columns

Index([u'Protein', u'Mutation(s)_PDB', u'Mutation(s)_cleaned', u'Location(s)',
       u'Hold_out_type', u'Hold_out_proteins', u'Affinity_mut (M)',
       u'Affinity_wt (M)', u'DDG', u'Reference', u'Protein 1', u'Protein 2',
       u'Temperature 298.15', u'Temperature Num', u'Temperature',
       u'kon_mut (M^(-1)s^(-1))', u'kon_wt (M^(-1)s^(-1))',
       u'koff_mut (s^(-1))', u'koff_wt (s^(-1))', u'dH_mut (kcal mol^(-1))',
       u'dH_wt (kcal mol^(-1))', u'dS_mut (cal mol^(-1) K^(-1))',
       u'dS_wt (cal mol^(-1) K^(-1))', u'Notes'],
      dtype='object')

In [3]:
skempi_df[skempi_df.Protein=="2VLJ_ABC_DE"]

,Protein,Mutation(s)_PDB,Mutation(s)_cleaned,Location(s),Hold_out_type,Hold_out_proteins,Affinity_mut (M),Affinity_wt (M),DDG,Reference,...,Temperature,kon_mut (M^(-1)s^(-1)),kon_wt (M^(-1)s^(-1)),koff_mut (s^(-1)),koff_wt (s^(-1)),dH_mut (kcal mol^(-1)),dH_wt (kcal mol^(-1)),dS_mut (cal mol^(-1) K^(-1)),dS_wt (cal mol^(-1) K^(-1)),Notes
2703,2VLJ_ABC_DE,DE32A,DE28A,COR,AB,AB,0.000074,0.000005,1.573201,18275829,...,298,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2704,2VLJ_ABC_DE,IE53V,IE49V,COR,AB,AB,0.000007,0.000005,0.200969,18275829,...,298,17100.0,31000.0,0.13,0.16,-22.0,-23.0,-50.3,-50.3,NaN
2705,2VLJ_ABC_DE,IE53L,IE49L,COR,AB,AB,0.000057,0.000005,1.418563,18275829,...,298,7000.0,31000.0,0.40,0.16,NaN,NaN,NaN,NaN,NaN
2706,2VLJ_ABC_DE,NE55A,NE51A,RIM,AB,AB,0.000035,0.000005,1.129622,18275829,...,298,18000.0,31000.0,0.63,0.16,-18.0,-23.0,-40.2,-50.3,NaN
2707,2VLJ_ABC_DE,NE55D,NE51D,RIM,AB,AB,0.000012,0.000005,0.495437,18275829,...,298,32300.0,31000.0,0.38,0.16,NaN,NaN,NaN,NaN,NaN
2708,2VLJ_ABC_DE,DE56A,DE52A,COR,AB,AB,0.000006,0.000005,0.132202,18275829,...,298,38000.0,31000.0,0.25,0.16,-24.0,-23.0,-57.0,-50.3,NaN
2709,2VLJ_ABC_DE,QE58A,QE54A,COR,AB,AB,0.000012,0.000005,0.495437,18275829,...,298,28000.0,31000.0,0.34,0.16,-18.0,-23.0,-36.9,-50.3,NaN
2710,2VLJ_ABC_DE,QE58E,QE54E,COR,AB,AB,0.000005,0.000005,0.000000,18275829,...,298,50000.0,31000.0,0.26,0.16,-20.0,-23.0,-43.6,-50.3,NaN
2711,2VLJ_ABC_DE,SE99A,SE95A,SUP,AB,AB,0.000005,0.000005,-0.035206,18275829,...,298,59000.0,31000.0,0.29,0.16,-18.0,-23.0,-33.5,-50.3,NaN
2712,2VLJ_ABC_DE,YE101A,YE97A,COR,AB,AB,0.000008,0.000005,0.232574,18275829,...,298,NaN,NaN,NaN,NaN,-29.0,-23.0,-73.8,-50.3,NaN


In [4]:
pdb_and_chains = set([tuple(pdb_str.split('_')) for pdb_str in skempi_df.Protein.values])

In [5]:
from skempi import *
import skempi as skempi
skempi.PDB_PATH = "../data/pdbs"

In [535]:
class Stride(object):

    def __init__(self, pdb):
        df = pd.read_csv('../data/stride/%s.out' % pdb)
        self._dict = {}
        for i, row in df.iterrows():
            self._dict[(row.Chain, row.Res-1)] = row.to_dict()

    def __getitem__(self, t):
        chain, res = t
        return self._dict[(chain, res)]
    
class SkempiRecord(object):
    def __init__(self, pdb, chains_a, chains_b):
        fd = open(osp.join(PDB_PATH, "%s.pdb" % pdb), 'r')
        self.struct = parse_pdb(pdb, fd)
        self.pdb = pdb
        self.chains_a = {c: self.struct[c] for c in chains_a}
        self.chains_b = {c: self.struct[c] for c in chains_b}
        self.res_chain_to_atom_indices = {}
        self.atom_indices_to_chain_res = {}
        self.atoms = []
        self.init_dictionaries()
        self.dist_mat = None
        self._profiles = {}
        self.init_profiles()
        self._stride = Stride(self.pdb)

    @property
    def chains(self):
        return self.struct.chains

    def init_profiles(self):
        self._profiles = {c: Profile(self.pdb, c) for c in self.chains}

    def get_profile(self, chain_id):
        return self._profiles[chain_id]

    @property
    def stride(self):
        return self._stride

    def init_dictionaries(self):
        for chain in self.struct:
            for res_i, res in enumerate(chain):
                for atom in res:
                    chain_id = chain.chain_id
                    if (chain_id, res_i) in self.res_chain_to_atom_indices:
                        self.res_chain_to_atom_indices[(chain_id, res_i)].append(len(self.atoms))
                    else:
                        self.res_chain_to_atom_indices[(chain_id, res_i)] = [len(self.atoms)]
                    self.atom_indices_to_chain_res[len(self.atoms)] = (chain_id, res_i)
                    self.atoms.append(atom)

    def compute_dist_mat(self):
        atoms = self.atoms
        self.dist_mat = get_distance_matrix(atoms1=atoms, atoms2=atoms)

    def __getitem__(self, chain):
        return self.chains[chain]

    def get_sphere_indices(self, chain, res_i, threshold):
        mat = self.dist_mat
        row_indices = self.res_chain_to_atom_indices[(chain, res_i)]
        col_indices = []
        for row_i in row_indices:
            col_indices.extend([ix for ix in np.where(mat[row_i] <= threshold)[0]])
        return set([self.atom_indices_to_chain_res[col_i] for col_i in col_indices])

    def get_stride(self):
        pass

    def __iter__(self):
        for chain in self.chains.values():
            yield chain

    def to_fasta(self):
        pass

In [536]:
record = SkempiRecord("2VLJ", "ABC", "DE")
# record = SkempiRecord("1CSE", "E", "I")

In [537]:
record.chains_a['A'].seq

'GSHSMRYFFTSVSRPGRGEPRFIAVGYVDDTQFVRFDSDAASQRMEPRAPWIEQEGPEYWDGETRKVKAHSQTHRVDLGTLRGYYNQSEAGSHTVQRMYGCDVGSDWRFLRGYHQYAYDGKDYIALKEDLRSWTAADMAAQTTKHKWEAAHVAEQLRAYLEGTCVEWLRRYLENGKETLQRTDAPKTHMTHHAVSDHEATLRCWALSFYPAEITLTWQRDGEDQTQDTELVETRPAGDGTFQKWAAVVVPSGQEQRYTCHVQHEGLPKPLTLRWEP'

In [538]:
record.compute_dist_mat()
sphere_ix = record.get_sphere_indices('A', 2, 6)
print(sphere_ix)
print(record.get_profile('A')[(2, 'A')])

set([('A', 1), ('A', 27), ('A', 0), ('A', 179), ('A', 100), ('A', 178), ('A', 103), ('A', 106), ('A', 102), ('A', 171), ('A', 101), ('A', 167), ('A', 4), ('A', 3), ('A', 104), ('A', 2), ('A', 28)])
0.00423728813559


In [540]:
record.stride[('A', 0)]

{'AA': 'GLY',
 'ASA': 52.1,
 'ASA_Chain': 52.1,
 'Chain': 'A',
 'Phi': 360.0,
 'Psi': -173.62,
 'Res': 1,
 'SS': 'C'}

In [541]:
record.dist_mat.shape

(6618, 6618)

In [542]:
from aaindex import *
B = BLOSUM62
C = SKOJ970101

def EI(m, w, P, i, B):
    return sum([P[(i, a)] * (B[(a, m)] - B[(a, w)]) for a in amino_acids])

def CP(mut, skempi, C, radius=6):
    
    i, chain_a = mut.i, mut.chain_id
    m, w = mut.m, mut.w
    
    def helper(P, j):
        return sum([P[(j, a)] * (C[(a, m)] - C[(a, w)]) for a in amino_acids])
    
    ret = 0
    for chain_b, j in skempi.get_sphere_indices(chain_a, i,radius):
        
        if j == i and chain_b == chain_a:
            a = skempi[chain_b][j].name
            assert a == w
            continue
        
        P = skempi.get_profile(chain_b) 
        
        ret += helper(P, j)
    
    return ret


# def CP(mut, skempi, C, radius=6):
    
#     i, chain_a = mut.i, mut.chain_id
#     m, w = mut.m, mut.w
    
#     ret = 0
#     for chain_b, j in skempi.get_sphere_indices(chain_a, i,radius):
        
#         a = skempi[chain_b][j].name
        
#         assert a == w
                
#         ret += C[(a, m)] - C[(a, w)]
    
#     return ret


class Mutation(object):
    
    def __init__(self, mutation):
        try:
            self.w = mutation[0]
            self.chain_id = mutation[1]
            self.i = int(mutation[2:-1]) - 1
            self.m = mutation[-1]
            self.ins_code = None

        except ValueError:
            self.w = mutation[0]
            self.chain_id = mutation[1]
            self.i = int(mutation[2:-2]) - 1
            self.m = mutation[-1]
            self.ins_code = mutation[-2]
            
    def __str__(self):
        return str(vars(self))


In [543]:
skempi_df = pd.read_excel(osp.join('../data', 'SKEMPI_1.1.xlsx'))

prots = skempi_df.Protein.values
skempi_records = {}

for t in tqdm(set([tuple(pdb_str.split('_')) for pdb_str in prots]),
              desc="skempi entries processed"):
    skempi_records[t] = SkempiRecord(*t)







skempi entries processed:   0%|          | 0/158 [00:00<?, ?it/s]





skempi entries processed:   1%|          | 1/158 [00:00<00:17,  8.82it/s]





skempi entries processed:   1%|▏         | 2/158 [00:00<00:17,  8.79it/s]





skempi entries processed:   2%|▏         | 3/158 [00:00<00:18,  8.48it/s]





skempi entries processed:   3%|▎         | 4/158 [00:00<00:18,  8.43it/s]





skempi entries processed:   3%|▎         | 5/158 [00:00<00:18,  8.28it/s]





skempi entries processed:   4%|▍         | 7/158 [00:00<00:16,  8.95it/s]





skempi entries processed:   6%|▌         | 9/158 [00:00<00:16,  9.01it/s]





skempi entries processed:   7%|▋         | 11/158 [00:01<00:15,  9.44it/s]





skempi entries processed:   8%|▊         | 12/158 [00:01<00:15,  9.31it/s]





skempi entries processed:   8%|▊         | 13/158 [00:01<00:15,  9.35it/s]





skempi entries processed:   9%|▉         | 15/158 [00:01<00:15,  9.44it/s]





skempi entries processed:  10%|█         | 16/158 

In [12]:
def comp_ei(mut, skempi_record, B, radius):
    P = skempi_record.get_profile(mut.chain_id)
    return EI(mut.m, mut.w, P, mut.i, B)

def comp_cp(mut, skempi_record, C, radius):
    return CP(mut, skempi_record, C, radius)


def get_ddg_ei_cp_arrays(M, func, radius=None):
    arr_ddg = []
    arr_obs = []
    pbar = tqdm(range(len(skempi_df)), desc="row processed")
    for i, row in skempi_df.iterrows():
        ddg = row.DDG
        arr_ddg.append(ddg)
        arr_obs_mut = []
        for mutation in row["Mutation(s)_cleaned"].split(','):
            mut = Mutation(mutation)
            t = tuple(row.Protein.split('_'))
            skempi_record = skempi_records[t]
            skempi_record.compute_dist_mat()
            obs = func(mut, skempi_record, M, radius)
            arr_obs_mut.append(obs)
        arr_obs.append(np.sum(arr_obs_mut))
        pbar.update(1)
    pbar.close()
    return arr_ddg, arr_obs

In [13]:
from scipy.stats import pearsonr

In [14]:
from itertools import product

def grid_search_cp(matrices=[SKOJ970101, BASU010101], radiuses=[4, 5, 6, 7, 8]):
    res_dict = {}
    for C, angs in product(matrices, radiuses):
        key = (str(C), angs)
        arr_ddg, arr_cp = get_ddg_ei_cp_arrays(C, comp_cp, angs)
        res_dict[key] = (arr_ddg, arr_cp)
        cor_cp = pearsonr(arr_ddg, arr_cp)
        print("%s: CP: %s" % (key, cor_cp,))
    return res_dict

def grid_search_ei(matrices=[BLOSUM62, SKOJ970101, BASU010101]):
    res_dict = {}
    for B in matrices:
        key = str(B)
        arr_ddg, arr_ei = get_ddg_ei_cp_arrays(B, comp_ei)
        res_dict[key] = (arr_ddg, arr_ei)
        cor_ei = pearsonr(arr_ddg, arr_ei)
        print("%s: EI: %s" % (key, cor_ei,))
    return res_dict


In [15]:
eis = grid_search_ei()

row processed:   0%|          | 1/3047 [00:00<05:30,  9.22it/s]

BLOSUM62: EI: (-0.18348266887605677, 1.7735594893088174e-24)


row processed:   0%|          | 1/3047 [00:00<05:06,  9.95it/s]

SKOJ970101: EI: (0.19216942446949448, 9.8286921678709035e-27)


row processed: 100%|██████████| 3047/3047 [08:52<00:00,  5.72it/s]

BASU010101: EI: (0.17535505843986804, 1.8228134582699703e-22)


In [16]:
# cps = grid_search_cp()

In [17]:
def comp_cp_a_b(mut, skempi_record, C, radius):
    return CP_A_B(mut, skempi_record, C, radius)


def get_ddg_cp_a_b_arrays(M, func, radius=None):
    arr_ddg = []
    arr_obs_a = []
    arr_obs_b = []
    pbar = tqdm(range(len(skempi_df)), desc="row processed")
    for i, row in skempi_df.iterrows():
        ddg = row.DDG
        arr_ddg.append(ddg)
        arr_obs_mut_a = []
        arr_obs_mut_b = []
        for mutation in row["Mutation(s)_cleaned"].split(','):
            mut = Mutation(mutation)
            t = tuple(row.Protein.split('_'))
            skempi_record = skempi_records[t]
            skempi_record.compute_dist_mat()
            obs_a, obs_b = func(mut, skempi_record, M, radius)
            arr_obs_mut_a.append(obs_a)
            arr_obs_mut_b.append(obs_b)
        arr_obs_a.append(np.sum(arr_obs_mut_a))
        arr_obs_b.append(np.sum(arr_obs_mut_b))
        pbar.update(1)
    pbar.close()
    return arr_ddg, arr_obs_a, arr_obs_b


def grid_search_cp_a_b(matrices=[SKOJ970101, BASU010101], radiuses=[4, 5, 6, 7, 8, 9, 10]):
    res_dict = {}
    for C, angs in product(matrices, radiuses):
        key = (str(C), angs)
        arr_ddg, arr_cp_a, arr_cp_b  = get_ddg_cp_a_b_arrays(C, comp_cp_a_b, angs)
        arr_cp = np.asarray(arr_cp_a) + np.asarray(arr_cp_b)
        res_dict[key] = (arr_ddg, arr_cp_a, arr_cp_b)
        cor_cp_a = pearsonr(arr_ddg, arr_cp_a)
        cor_cp_b = pearsonr(arr_ddg, arr_cp_b)
        cor_cp = pearsonr(arr_ddg, arr_cp)
        print("%s: CP_A: %s, CP_B: %s, CP %s" % (key, cor_cp_a, cor_cp_b, cor_cp))
    return res_dict

In [18]:
def CP_A_B(mut, skempi, C, radius=6):
    
    i, chain_a = mut.i, mut.chain_id
    m, w = mut.m, mut.w
    
#     def helper(P, j):
#         return sum([P[(j, a)] * (C[(a, m)] - C[(a, w)]) for a in amino_acids])
    
    def helper(a, j):
        return C[(a, m)] - C[(a, w)]
    
    retA, retB = 0, 0
    for chain_b, j in skempi.get_sphere_indices(chain_a, i,radius):

        a = skempi[chain_b][j].name
        if j == i and chain_b == chain_a:
            assert a == w
            continue
                
        P = skempi.get_profile(chain_b) 

        if chain_b == chain_a:  
            retA += helper(a, j)
        
        else:
            retB += helper(a, j)
    
    return retA, retB


In [19]:
cp_a_b_s_no_profile = grid_search_cp_a_b(matrices=[SKOJ970101, BASU010101], radiuses=[6, 7])

row processed:   0%|          | 2/3047 [00:00<04:40, 10.85it/s]

('SKOJ970101', 6): CP_A: (0.12899241387456756, 8.8507553354571917e-13), CP_B: (0.31510033967058948, 3.2909472514044773e-71), CP (0.23025093858330695, 5.9899593202034257e-38)


row processed:   0%|          | 2/3047 [00:00<04:19, 11.75it/s]

('SKOJ970101', 7): CP_A: (0.12813871212570935, 1.2512498088379469e-12), CP_B: (0.30767202856302794, 8.2220457876862489e-68), CP (0.22746400267848199, 4.7068139278480544e-37)


row processed:   0%|          | 2/3047 [00:00<04:37, 10.98it/s]

('BASU010101', 6): CP_A: (0.1126933207499565, 4.4332068307731485e-10), CP_B: (0.35501818492812953, 3.3990017934275672e-91), CP (0.24516152877860625, 6.0580342404484527e-43)


row processed: 100%|██████████| 3047/3047 [08:44<00:00,  5.81it/s]

('BASU010101', 7): CP_A: (0.13643805479717855, 3.9195895664075284e-14), CP_B: (0.33861062950737597, 1.2780416763987716e-82), CP (0.25483942919026048, 2.2527136127542865e-46)


In [20]:
def CP_A_B(mut, skempi, C, radius=6):
    
    i, chain_a = mut.i, mut.chain_id
    m, w = mut.m, mut.w
    
    def helper(P, j):
        return sum([P[(j, a)] * (C[(a, m)] - C[(a, w)]) for a in amino_acids])
    
#     def helper(a, j):
#         return C[(a, m)] - C[(a, w)]
    
    retA, retB = 0, 0
    for chain_b, j in skempi.get_sphere_indices(chain_a, i,radius):

        a = skempi[chain_b][j].name
        if j == i and chain_b == chain_a:
            assert a == w
            continue
                
        P = skempi.get_profile(chain_b) 

        if chain_b == chain_a:  
            retA += helper(P, j)
        
        else:
            retB += helper(P, j)
    
    return retA, retB

In [21]:
cp_a_b_s_orig = grid_search_cp_a_b(matrices=[SKOJ970101, BASU010101], radiuses=[6, 7])

row processed:   0%|          | 2/3047 [00:00<04:33, 11.12it/s]

('SKOJ970101', 6): CP_A: (0.15922361393701534, 9.4069290943909113e-19), CP_B: (0.28193363413720945, 8.7942190111787501e-57), CP (0.23091446921072051, 3.6517239867096613e-38)


row processed:   0%|          | 2/3047 [00:00<04:33, 11.12it/s]

('SKOJ970101', 7): CP_A: (0.15680392577872013, 3.1514065258845439e-18), CP_B: (0.2977055920083635, 2.0945759996836172e-63), CP (0.23752569390737782, 2.4195946509145309e-40)


row processed:   0%|          | 2/3047 [00:00<04:19, 11.73it/s]

('BASU010101', 6): CP_A: (0.16604593209712934, 2.8085223686026527e-20), CP_B: (0.3424606427389556, 1.381754790590439e-84), CP (0.26530083607511301, 2.9946474562890449e-50)


row processed: 100%|██████████| 3047/3047 [08:50<00:00,  5.75it/s]

('BASU010101', 7): CP_A: (0.17621764015649943, 1.1265247125320121e-22), CP_B: (0.34222839700718038, 1.8189732609036253e-84), CP (0.27527019137816483, 4.1318263822884498e-54)


In [22]:
def CP_A_B(mut, skempi, C, radius=6):
    
    i, chain_a = mut.i, mut.chain_id
    m, w = mut.m, mut.w
    
    def helper(P, j):
        return sum([0.05 * (C[(a, m)] - C[(a, w)]) for a in amino_acids])
    
#     def helper(a, j):
#         return C[(a, m)] - C[(a, w)]
    
    retA, retB = 0, 0
    for chain_b, j in skempi.get_sphere_indices(chain_a, i,radius):

        a = skempi[chain_b][j].name
        if j == i and chain_b == chain_a:
            assert a == w
            continue
                
        P = skempi.get_profile(chain_b) 

        if chain_b == chain_a:  
            retA += helper(P, j)
        
        else:
            retB += helper(P, j)
    
    return retA, retB

In [23]:
cp_a_b_s_uniform = grid_search_cp_a_b(matrices=[SKOJ970101, BASU010101], radiuses=[6, 7])

row processed:   0%|          | 2/3047 [00:00<04:19, 11.73it/s]

('SKOJ970101', 6): CP_A: (0.16123595170241231, 3.3922690400735878e-19), CP_B: (0.26672526849365225, 8.6058008697708072e-51), CP (0.2251320913909437, 2.586235766630754e-36)


row processed:   0%|          | 1/3047 [00:00<05:42,  8.88it/s]

('SKOJ970101', 7): CP_A: (0.1620582028142715, 2.2276542254997738e-19), CP_B: (0.29191128666928023, 6.3620596148568711e-61), CP (0.23883741861785063, 8.7773810986036118e-41)


row processed:   0%|          | 2/3047 [00:00<04:11, 12.10it/s]

('BASU010101', 6): CP_A: (0.21300246807984247, 1.3416778766703381e-32), CP_B: (0.34737111879464089, 3.9086171731054629e-87), CP (0.29624906619169722, 8.9240722429331053e-63)


row processed: 100%|██████████| 3047/3047 [08:52<00:00,  5.72it/s]

('BASU010101', 7): CP_A: (0.21351384288462372, 9.4522018304968719e-33), CP_B: (0.36314745255523262, 1.2296267842765852e-95), CP (0.30740335992410711, 1.0865183409662947e-67)


In [24]:
cp_a_b_s_uniform.keys(), cp_a_b_s_orig.keys(), cp_a_b_s_no_profile.keys(), eis.keys()

([('SKOJ970101', 7), ('BASU010101', 7), ('BASU010101', 6), ('SKOJ970101', 6)],
 [('SKOJ970101', 7), ('BASU010101', 7), ('BASU010101', 6), ('SKOJ970101', 6)],
 [('SKOJ970101', 7), ('BASU010101', 7), ('BASU010101', 6), ('SKOJ970101', 6)],
 ['BLOSUM62', 'SKOJ970101', 'BASU010101'])

In [544]:
len(cp_a_b_s_uniform[('SKOJ970101', 7)])

all_features = {}

def register_cp_a_b(cp_a_b, prefix):
    for key, val in cp_a_b.iteritems():
        _, cp_a, cp_b = val
        mat, rad = key
        all_features[(prefix, "CP_A", mat, rad)] = cp_a
        all_features[(prefix, "CP_B", mat, rad)] = cp_b
        
def register_eis(eis):
    for key, val in eis.iteritems():
        _, ei = val
        all_features[("EI", key)] = ei

In [545]:
register_cp_a_b(cp_a_b_s_uniform, "uniform")
register_cp_a_b(cp_a_b_s_orig, "original")
register_cp_a_b(cp_a_b_s_no_profile, "no_profile")
register_eis(eis)

In [546]:
num_muts = np.asarray([len(mut.split(",")) for mut in skempi_df["Mutation(s)_cleaned"]])
pearsonr(skempi_df.DDG, np.log(num_muts)), pearsonr(skempi_df.DDG, num_muts)

((0.21924597848778335, 1.7513323606150017e-34),
 (0.13427861416945261, 9.8559457179327132e-14))

In [547]:
all_features["#mutations"] = np.log(num_muts)

In [717]:
def get_stride_array(func, agg=np.sum):
    arr_stride = []
    pbar = tqdm(range(len(skempi_df)), desc="row processed")
    for i, row in skempi_df.iterrows():
        arr_obs_mut = []
        for mutation in row["Mutation(s)_cleaned"].split(','):
            mut = Mutation(mutation)
            res_i, chain_id = mut.i, mut.chain_id
            t = tuple(row.Protein.split('_'))
            skempi_record = skempi_records[t]
            stride = skempi_record.stride[(chain_id, res_i)]
            skempi_record.compute_dist_mat()
            obs = func(stride)
            arr_obs_mut.append(obs)
        arr_stride.append(agg(arr_obs_mut))
        pbar.update(1)
    pbar.close()
    return arr_stride

In [710]:
def asa_diff(stride):
    return abs(stride["ASA"] - stride["ASA_Chain"])

stride_arr = get_stride_array(asa_diff)











row processed:   0%|          | 0/3047 [00:00<?, ?it/s]









row processed:   0%|          | 1/3047 [00:00<07:08,  7.11it/s]









row processed:   0%|          | 2/3047 [00:00<06:41,  7.59it/s]

(1,)
(2,)












row processed:   0%|          | 4/3047 [00:00<05:59,  8.46it/s]









row processed:   0%|          | 5/3047 [00:00<05:48,  8.73it/s]

(3,)
(4,)
(5,)












row processed:   0%|          | 6/3047 [00:00<05:38,  8.98it/s]









row processed:   0%|          | 8/3047 [00:00<05:07,  9.88it/s]

...











row processed:  14%|█▍        | 420/3047 [00:57<22:01,  1.99it/s]









row processed:  14%|█▍        | 423/3047 [00:57<15:58,  2.74it/s]

(420,)
(421,)
(422,)
(423,)
(424,)












row processed:  14%|█▍        | 425/3047 [00:57<11:54,  3.67it/s]

(425,)


KeyboardInterrupt: 

In [714]:
all_features["abs(ASA-ASA_Chain)"] = stride_arr
pearsonr(skempi_df.DDG, stride_arr)

(0.44807091830983697, 2.1281783251917486e-150)

In [718]:
DSSP = ["G", "H", "I", "T", "E", "B", "S", "C"]

from sklearn import preprocessing
lb = preprocessing.LabelBinarizer()

lb.fit(DSSP) 

def get_bin_ss(stride):
    return lb.transform([stride["SS"]])[0]

In [731]:
ss_arr = get_stride_array(get_bin_ss, agg=lambda a: np.sum(a, axis=0))














row processed:   0%|          | 0/3047 [00:00<?, ?it/s]












row processed:   0%|          | 2/3047 [00:00<04:31, 11.20it/s]












row processed:   0%|          | 3/3047 [00:00<04:45, 10.66it/s]












row processed:   0%|          | 5/3047 [00:00<04:42, 10.78it/s]












row processed:   0%|          | 7/3047 [00:00<04:32, 11.16it/s]












row processed:   0%|          | 9/3047 [00:00<04:26, 11.39it/s]












row processed:   0%|          | 11/3047 [00:00<04:28, 11.31it/s]












row processed:   0%|          | 13/3047 [00:01<04:23, 11.51it/s]












row processed:   0%|          | 15/3047 [00:01<04:34, 11.04it/s]












row processed:   1%|          | 17/3047 [00:01<04:53, 10.34it/s]












row processed:   1%|          | 19/3047 [00:01<05:00, 10.06it/s]












row processed:   1%|          | 20/3047 [00:01<06:33,  7.70it/s]












row processed:   1%|          | 21/3047 [00:02<06:15,  8.05it/s]














row processed:   9%|▊         | 263/3047 [00:25<15:38,  2.97it/s]












row processed:   9%|▊         | 264/3047 [00:25<15:14,  3.04it/s]












row processed:   9%|▊         | 265/3047 [00:25<14:41,  3.16it/s]












row processed:   9%|▊         | 266/3047 [00:26<14:17,  3.24it/s]












row processed:   9%|▉         | 267/3047 [00:26<14:09,  3.27it/s]












row processed:   9%|▉         | 269/3047 [00:26<10:43,  4.32it/s]












row processed:   9%|▉         | 271/3047 [00:26<08:13,  5.63it/s]












row processed:   9%|▉         | 273/3047 [00:26<07:20,  6.29it/s]












row processed:   9%|▉         | 275/3047 [00:27<07:20,  6.29it/s]












row processed:   9%|▉         | 276/3047 [00:27<07:33,  6.11it/s]












row processed:   9%|▉         | 277/3047 [00:27<07:32,  6.12it/s]












row processed:   9%|▉         | 278/3047 [00:27<07:36,  6.07it/s]












row processed:   9%|▉         | 279/3047 [00:27<07:34,  6.09it/s

row processed:  16%|█▌        | 492/3047 [00:58<10:24,  4.09it/s]












row processed:  16%|█▌        | 493/3047 [00:58<09:07,  4.66it/s]












row processed:  16%|█▌        | 494/3047 [00:58<08:09,  5.21it/s]












row processed:  16%|█▌        | 495/3047 [00:58<07:30,  5.67it/s]












row processed:  16%|█▋        | 496/3047 [00:58<07:06,  5.99it/s]












row processed:  16%|█▋        | 497/3047 [00:58<06:48,  6.24it/s]












row processed:  16%|█▋        | 498/3047 [00:59<06:37,  6.41it/s]












row processed:  16%|█▋        | 499/3047 [00:59<07:17,  5.82it/s]












row processed:  16%|█▋        | 500/3047 [00:59<07:43,  5.50it/s]












row processed:  16%|█▋        | 501/3047 [00:59<08:00,  5.30it/s]












row processed:  16%|█▋        | 502/3047 [00:59<08:07,  5.22it/s]












row processed:  17%|█▋        | 503/3047 [01:00<08:14,  5.14it/s]












row processed:  17%|█▋        | 504/3047 [01:00<08:23,  5.05it/s

row processed:  22%|██▏       | 684/3047 [01:29<02:15, 17.48it/s]












row processed:  23%|██▎       | 687/3047 [01:29<02:01, 19.37it/s]












row processed:  23%|██▎       | 691/3047 [01:29<01:46, 22.05it/s]












row processed:  23%|██▎       | 695/3047 [01:29<01:35, 24.65it/s]












row processed:  23%|██▎       | 699/3047 [01:29<01:27, 26.79it/s]












row processed:  23%|██▎       | 703/3047 [01:30<02:49, 13.83it/s]












row processed:  23%|██▎       | 706/3047 [01:30<03:59,  9.77it/s]












row processed:  23%|██▎       | 708/3047 [01:30<04:19,  9.02it/s]












row processed:  23%|██▎       | 710/3047 [01:31<05:56,  6.55it/s]












row processed:  23%|██▎       | 712/3047 [01:31<05:43,  6.81it/s]












row processed:  23%|██▎       | 714/3047 [01:32<06:13,  6.25it/s]












row processed:  23%|██▎       | 715/3047 [01:32<07:20,  5.29it/s]












row processed:  23%|██▎       | 716/3047 [01:32<08:09,  4.76it/s

row processed:  28%|██▊       | 859/3047 [02:17<05:40,  6.42it/s]












row processed:  28%|██▊       | 860/3047 [02:18<05:37,  6.48it/s]












row processed:  28%|██▊       | 861/3047 [02:18<05:36,  6.49it/s]












row processed:  28%|██▊       | 862/3047 [02:18<05:33,  6.56it/s]












row processed:  28%|██▊       | 863/3047 [02:18<05:34,  6.54it/s]












row processed:  28%|██▊       | 864/3047 [02:18<05:33,  6.56it/s]












row processed:  28%|██▊       | 865/3047 [02:18<05:32,  6.56it/s]












row processed:  28%|██▊       | 866/3047 [02:19<05:28,  6.64it/s]












row processed:  28%|██▊       | 867/3047 [02:19<05:24,  6.72it/s]












row processed:  28%|██▊       | 868/3047 [02:19<05:21,  6.77it/s]












row processed:  29%|██▊       | 869/3047 [02:19<05:20,  6.79it/s]












row processed:  29%|██▊       | 870/3047 [02:19<05:25,  6.68it/s]












row processed:  29%|██▊       | 871/3047 [02:19<06:57,  5.22it/s

...

row processed:  67%|██████▋   | 2053/3047 [06:06<01:01, 16.29it/s]












row processed:  67%|██████▋   | 2055/3047 [06:06<01:01, 16.12it/s]












row processed:  68%|██████▊   | 2057/3047 [06:06<01:01, 16.20it/s]












row processed:  68%|██████▊   | 2059/3047 [06:07<00:59, 16.47it/s]












row processed:  68%|██████▊   | 2061/3047 [06:07<00:59, 16.63it/s]












row processed:  68%|██████▊   | 2063/3047 [06:07<00:58, 16.79it/s]












row processed:  68%|██████▊   | 2065/3047 [06:07<00:58, 16.88it/s]












row processed:  68%|██████▊   | 2067/3047 [06:07<00:58, 16.89it/s]












row processed:  68%|██████▊   | 2069/3047 [06:07<00:57, 17.00it/s]












row processed:  68%|██████▊   | 2071/3047 [06:07<01:01, 15.88it/s]












row processed:  68%|██████▊   | 2073/3047 [06:08<01:07, 14.33it/s]












row processed:  68%|██████▊   | 2075/3047 [06:08<01:13, 13.30it/s]












row processed:  68%|██████▊   | 2077/3047 [06:08<01:

row processed:  77%|███████▋  | 2359/3047 [06:26<00:47, 14.39it/s]












row processed:  77%|███████▋  | 2361/3047 [06:26<00:50, 13.62it/s]












row processed:  78%|███████▊  | 2363/3047 [06:26<00:47, 14.46it/s]












row processed:  78%|███████▊  | 2365/3047 [06:26<00:45, 15.08it/s]












row processed:  78%|███████▊  | 2367/3047 [06:26<00:42, 16.16it/s]












row processed:  78%|███████▊  | 2369/3047 [06:27<00:40, 16.66it/s]












row processed:  78%|███████▊  | 2371/3047 [06:27<00:40, 16.82it/s]












row processed:  78%|███████▊  | 2373/3047 [06:27<00:41, 16.43it/s]












row processed:  78%|███████▊  | 2375/3047 [06:27<00:44, 15.24it/s]












row processed:  78%|███████▊  | 2377/3047 [06:27<00:47, 13.97it/s]












row processed:  78%|███████▊  | 2379/3047 [06:27<00:49, 13.46it/s]












row processed:  78%|███████▊  | 2381/3047 [06:27<00:51, 12.92it/s]












row processed:  78%|███████▊  | 2383/3047 [06:28<00:

row processed:  86%|████████▌ | 2618/3047 [06:52<02:03,  3.46it/s]












row processed:  86%|████████▌ | 2619/3047 [06:53<03:07,  2.29it/s]












row processed:  86%|████████▌ | 2620/3047 [06:54<02:44,  2.60it/s]












row processed:  86%|████████▌ | 2621/3047 [06:54<02:27,  2.88it/s]












row processed:  86%|████████▌ | 2622/3047 [06:54<02:51,  2.48it/s]












row processed:  86%|████████▌ | 2623/3047 [06:55<02:33,  2.76it/s]












row processed:  86%|████████▌ | 2624/3047 [06:55<02:23,  2.95it/s]












row processed:  86%|████████▌ | 2625/3047 [06:55<02:13,  3.15it/s]












row processed:  86%|████████▌ | 2626/3047 [06:56<02:38,  2.66it/s]












row processed:  86%|████████▌ | 2627/3047 [06:56<02:55,  2.40it/s]












row processed:  86%|████████▌ | 2628/3047 [06:57<03:06,  2.25it/s]












row processed:  86%|████████▋ | 2629/3047 [06:57<02:43,  2.55it/s]












row processed:  86%|████████▋ | 2630/3047 [06:57<02:

row processed:  91%|█████████▏| 2785/3047 [07:38<02:00,  2.18it/s]












row processed:  91%|█████████▏| 2786/3047 [07:38<01:33,  2.79it/s]












row processed:  91%|█████████▏| 2787/3047 [07:38<01:18,  3.30it/s]












row processed:  91%|█████████▏| 2788/3047 [07:38<01:05,  3.97it/s]












row processed:  92%|█████████▏| 2789/3047 [07:39<01:07,  3.82it/s]












row processed:  92%|█████████▏| 2791/3047 [07:39<00:51,  4.94it/s]












row processed:  92%|█████████▏| 2793/3047 [07:39<00:46,  5.48it/s]












row processed:  92%|█████████▏| 2795/3047 [07:39<00:36,  6.82it/s]












row processed:  92%|█████████▏| 2797/3047 [07:39<00:32,  7.71it/s]












row processed:  92%|█████████▏| 2799/3047 [07:39<00:27,  9.18it/s]












row processed:  92%|█████████▏| 2801/3047 [07:39<00:23, 10.36it/s]












row processed:  92%|█████████▏| 2803/3047 [07:40<00:23, 10.56it/s]












row processed:  92%|█████████▏| 2805/3047 [07:40<00:

row processed:  97%|█████████▋| 2951/3047 [08:24<00:17,  5.43it/s]












row processed:  97%|█████████▋| 2952/3047 [08:24<00:18,  5.17it/s]












row processed:  97%|█████████▋| 2953/3047 [08:25<00:22,  4.25it/s]












row processed:  97%|█████████▋| 2954/3047 [08:25<00:24,  3.83it/s]












row processed:  97%|█████████▋| 2955/3047 [08:25<00:21,  4.24it/s]












row processed:  97%|█████████▋| 2956/3047 [08:25<00:21,  4.27it/s]












row processed:  97%|█████████▋| 2957/3047 [08:26<00:22,  4.05it/s]












row processed:  97%|█████████▋| 2958/3047 [08:26<00:25,  3.56it/s]












row processed:  97%|█████████▋| 2959/3047 [08:26<00:27,  3.18it/s]












row processed:  97%|█████████▋| 2960/3047 [08:27<00:26,  3.28it/s]












row processed:  97%|█████████▋| 2961/3047 [08:27<00:25,  3.34it/s]












row processed:  97%|█████████▋| 2962/3047 [08:27<00:26,  3.16it/s]












row processed:  97%|█████████▋| 2963/3047 [08:28<00:

In [732]:
[pearsonr(skempi_df.DDG, np.asarray(ss_arr)[:, j]) for j in range(8)]

[(0.046420830918097108, 0.010384883923705844),
 (0.19428201978462795, 2.6734087103648955e-27),
 (0.13355078837182441, 1.3403716975243515e-13),
 (0.020648461720111762, 0.25451909563207237),
 (-0.041496984072534521, 0.021982781352113706),
 (nan, 1.0),
 (nan, 1.0),
 (0.041696500814343597, 0.021352756381487238)]

In [733]:
all_features.keys(), len(all_features.keys())

([('uniform', 'CP_B', 'SKOJ970101', 6),
  '#mutations',
  ('original', 'CP_B', 'SKOJ970101', 7),
  ('no_profile', 'CP_A', 'SKOJ970101', 6),
  ('original', 'CP_A', 'SKOJ970101', 7),
  ('no_profile', 'CP_A', 'BASU010101', 7),
  ('uniform', 'CP_A', 'SKOJ970101', 7),
  ('no_profile', 'CP_B', 'SKOJ970101', 6),
  ('original', 'CP_A', 'BASU010101', 6),
  ('uniform', 'CP_B', 'BASU010101', 6),
  ('original', 'CP_B', 'BASU010101', 6),
  ('no_profile', 'CP_B', 'SKOJ970101', 7),
  ('original', 'CP_B', 'SKOJ970101', 6),
  ('no_profile', 'CP_A', 'SKOJ970101', 7),
  ('EI', 'SKOJ970101'),
  ('no_profile', 'CP_A', 'BASU010101', 6),
  ('uniform', 'CP_A', 'BASU010101', 7),
  ('no_profile', 'CP_B', 'BASU010101', 6),
  ('EI', 'BLOSUM62'),
  ('original', 'CP_B', 'BASU010101', 7),
  ('uniform', 'CP_B', 'BASU010101', 7),
  ('uniform', 'CP_B', 'SKOJ970101', 7),
  'abs(ASA-ASA_Chain)',
  ('EI', 'BASU010101'),
  ('original', 'CP_A', 'SKOJ970101', 6),
  ('uniform', 'CP_A', 'BASU010101', 6),
  ('uniform', 'CP_A', 

In [734]:
import itertools

In [735]:
xcor_mat = np.corrcoef(np.asarray(all_features.values()))

In [736]:
xcor_mat.shape

(29, 29)

In [737]:
class XCor(object):
    
    def __init__(self, all_features):
        self.feat_name_to_indx = {key:i for i, key in enumerate(all_features.keys())}
        self.xcor_mat = np.corrcoef(np.asarray(all_features.values()))
        
    def __getitem__(self, t):
        feat1, feat2 = t
        i = self.feat_name_to_indx[feat1]
        j = self.feat_name_to_indx[feat2]
        return self.xcor_mat[(i, j)]

In [738]:
xcor = XCor(all_features)

In [739]:
def search_min_xcor(all_features, th=0.05):
    acc = set()
    for comb in itertools.combinations(all_features.keys(), 2):
        feat1, feat2 = comb
        rho = xcor[(feat1, feat2)]
        if abs(rho) < th:
            print(feat1, feat2, rho)
            acc.add(feat1)
            acc.add(feat2)
    return acc

In [783]:
acc_feats = search_min_xcor(all_features)

(('uniform', 'CP_B', 'SKOJ970101', 6), '#mutations', 0.039810904106046208)
('#mutations', ('no_profile', 'CP_A', 'SKOJ970101', 6), -0.033714399623451741)
('#mutations', ('original', 'CP_A', 'BASU010101', 6), 0.017056502896153274)
('#mutations', ('no_profile', 'CP_B', 'SKOJ970101', 7), 0.042463711564533047)
('#mutations', ('no_profile', 'CP_A', 'SKOJ970101', 7), -0.038619918216982803)
('#mutations', ('EI', 'SKOJ970101'), -0.0011196272438495438)
('#mutations', ('uniform', 'CP_B', 'SKOJ970101', 7), 0.042943490507457799)
('#mutations', ('no_profile', 'CP_B', 'BASU010101', 7), 0.031059433777381178)
('#mutations', ('original', 'CP_A', 'BASU010101', 7), 0.046772400285923214)
(('no_profile', 'CP_A', 'SKOJ970101', 6), 'abs(ASA-ASA_Chain)', 0.049260535153685864)
(('uniform', 'CP_A', 'SKOJ970101', 7), 'abs(ASA-ASA_Chain)', 0.047348865688833862)
(('uniform', 'CP_B', 'BASU010101', 6), ('EI', 'BLOSUM62'), -0.017812503616760917)
(('EI', 'BLOSUM62'), ('uniform', 'CP_B', 'BASU010101', 7), -0.0275336087

In [784]:
len(acc_feats), acc_feats

(16,
 {'#mutations',
  'abs(ASA-ASA_Chain)',
  ('EI', 'BLOSUM62'),
  ('EI', 'SKOJ970101'),
  ('no_profile', 'CP_A', 'SKOJ970101', 6),
  ('no_profile', 'CP_A', 'SKOJ970101', 7),
  ('no_profile', 'CP_B', 'BASU010101', 7),
  ('no_profile', 'CP_B', 'SKOJ970101', 7),
  ('original', 'CP_A', 'BASU010101', 6),
  ('original', 'CP_A', 'BASU010101', 7),
  ('uniform', 'CP_A', 'SKOJ970101', 6),
  ('uniform', 'CP_A', 'SKOJ970101', 7),
  ('uniform', 'CP_B', 'BASU010101', 6),
  ('uniform', 'CP_B', 'BASU010101', 7),
  ('uniform', 'CP_B', 'SKOJ970101', 6),
  ('uniform', 'CP_B', 'SKOJ970101', 7)})

In [797]:
acc_feats = {'#mutations',
  'abs(ASA-ASA_Chain)',
  ('EI', 'BLOSUM62'),
  ('EI', 'SKOJ970101'),
  ('EI', 'BASU010101'),
  ('no_profile', 'CP_A', 'SKOJ970101', 6),
  ('no_profile', 'CP_A', 'SKOJ970101', 7),
  ('no_profile', 'CP_B', 'BASU010101', 7),
  ('no_profile', 'CP_B', 'SKOJ970101', 7),
  ('uniform', 'CP_A', 'SKOJ970101', 6),
  ('uniform', 'CP_A', 'SKOJ970101', 7),
  ('uniform', 'CP_B', 'BASU010101', 6),
  ('uniform', 'CP_B', 'BASU010101', 7),
  ('uniform', 'CP_B', 'SKOJ970101', 6),
  ('uniform', 'CP_B', 'SKOJ970101', 7)}

In [798]:
G1 = [
    "%s_%s_%s" % (s[:4], s[4], s[5].strip())  for s in  
    """
    1CSOEI 1CT0EI 1CT2EI 1CT4EI 1SGDEI 1SGEEI 1SGNEI 1SGPEI 1SGQEI 1SGYEI 2NU0EI
    2NU1EI 2NU2EI 2NU4EI 2SGPEI 2SGQEI 3SGBEI 1IARAB 1XD3AB 1F47AB 1ACBEI
    1H9DAB 2HRKAB 3BP8AC 2OOBAB
    """.split(' ') if s.strip()]

G2 = [
    "%s_%s_%s" % (s[:4], s[4], s[5].strip()) for s in  
    """
    1JTGAB 1S0WAC 2G2UAB 2G2WAB 1A4YAB 1Z7XWX 2GOXAB 2NOJAB 3D5RAC
    3D5SAC 1KACAB 1P69AB 1P6AAB 3BK3AC 1JCKAB 1SBBAB 4CPAAI 1S1QAB 2B42AB
    1E96AB 2I26NL
    """.split(' ') if s.strip()]


G3 = [
    "%s_%s_%s" % (s[:4], s[4], s[5].strip())  for s in  
    """
    1PPFEI 1CSEEI 1SBNEI 1TM1EI 1TM3EI 1TM4EI 1TM5EI 1TM7EI 1TMGEI 1TO1EI 1Y1KEI
    1Y33EI 1Y34EI 1Y3BEI 1Y4AEI 1GC1GC 2SICEI 2O3BAB 1FC2CD 2BTFAP 1EFNAB
    2A9KAB
    """.split(' ') if s.strip()]

G4 = [
    "%s_%s_%s" % (s[:4], s[4], s[5].strip())  for s in  
    """
    1R0REI 1EAWAB 2FTLEI 3BTDEI 3BTEEI 3BTFEI 3BTGEI 3BTHEI 3BTMEI 3BTQEI
    3BTTEI 3BTWEI 1AK4AD 1M9EAD 2J0TAD 1FFWAB 1MAHAF 1UUZAD 1SMFEI 2AJFAE
    2J1KCT
    """.split(' ') if s.strip()]

G5 = [
    "%s_%s_%s" % (s[:4], s[4], s[5].strip())  for s in  
    """
    1B2SAD 1B2UAD 1B3SAD 1BRSAD 1X1XAD 1EMVAB 2VLNAB 2VLOAB 2VLQAB
    2WPTAB 1A22AB 2B0ZAB 2B10AB 2B11AB 2B12AB 2PCBAB 2PCCAB 1KTZAB 1LFDAB
    1FCCAC 1GL0EI 1GL1AI 1HE8AB 2HLEAB 2I9BAE
    """.split(' ') if s.strip()]

G6 = [prot for prot in skempi_df.Protein if prot not in set(G1 + G2 + G3 + G4 + G5)]

In [799]:
sum(skempi_df.Protein.isin(G6))

983

In [800]:
df = skempi_df
from sklearn.preprocessing import StandardScaler

def run_cv_test(X, get_regressor, normalize=0):
    gt, preds = [], []
    for group in [G1, G2, G3, G4, G5]:
        indx_tst = df.Protein.isin(group)
        indx_trn = np.logical_not(indx_tst)
        y_trn = df.DDG[indx_trn]
        y_true = df.DDG[indx_tst]
        X_trn = X[indx_trn]
        X_tst = X[indx_tst]
        regressor = get_regressor()
        if normalize == 1:
            scaler = StandardScaler()
            scaler.fit(X_trn)
            X_trn, X_tst = scaler.transform(X_trn), scaler.transform(X_tst)
        regressor.fit(X_trn, y_trn)
        y_pred = regressor.predict(X_tst)
        print(pearsonr(y_true, y_pred))
        preds.extend(y_pred)
        gt.extend(y_true)
    return gt, preds

In [801]:
X = np.transpose([all_features[feat] for feat in acc_feats])
X = np.concatenate([X, np.asarray(ss_arr)], axis=1)
X.shape

(3047, 23)

In [802]:
from sklearn.ensemble import RandomForestRegressor
def get_regressor(): return RandomForestRegressor(n_estimators=100, random_state=101)
gt, preds = run_cv_test(X, get_regressor, normalize=1)
print(pearsonr(gt, preds))
len(gt)

(0.69888841990325246, 3.599609198720432e-59)
(0.53874098691958006, 6.537995574037864e-27)
(0.60198739783958222, 2.6194166429084624e-40)
(0.26846690437762311, 2.2909689268459242e-08)
(0.62223008385708112, 1.599771739340398e-56)
(0.49653652205471754, 6.1966484463430904e-129)


2064

In [803]:
from sklearn.ensemble import RandomForestRegressor
def get_regressor(): return SVR(kernel='rbf')
gt, preds = run_cv_test(X, get_regressor, normalize=1)
print(pearsonr(gt, preds))
len(gt)

(0.58022726828700433, 6.4478675035687233e-37)
(0.4895111455539598, 7.8999468588461493e-22)
(0.60088575024732327, 3.9460649653023566e-40)
(0.23440846061454071, 1.1889598147058405e-06)
(0.55356029414812391, 1.1001176820293134e-42)
(0.44084021868761042, 6.9509966485260807e-99)


2064

In [804]:
def run_cv_test(X, alpha=0.2, normalize=1):
    gt, preds = [], []
    for group in [G1, G2, G3, G4, G5]:
        indx_tst = df.Protein.isin(group)
        indx_trn = np.logical_not(indx_tst)
        y_trn = df.DDG[indx_trn]
        y_true = df.DDG[indx_tst]
        X_trn = X[indx_trn]
        X_tst = X[indx_tst]
        rf = RandomForestRegressor(n_estimators=50, random_state=101)
        svr = SVR(kernel='rbf')
        if normalize == 1:
            scaler = StandardScaler()
            scaler.fit(X_trn)
            X_trn, X_tst = scaler.transform(X_trn), scaler.transform(X_tst)
        svr.fit(X_trn, y_trn)
        rf.fit(X_trn, y_trn)
        y_pred_svr = svr.predict(X_tst)
        y_pred_rf = rf.predict(X_tst)
        y_pred = alpha * y_pred_svr + (1-alpha) * y_pred_rf
        print(pearsonr(y_true, y_pred))
        preds.extend(y_pred)
        gt.extend(y_true)
    return gt, preds

In [805]:
gt, preds = run_cv_test(X, normalize=1)
print(pearsonr(gt, preds))
len(gt)

(0.69352295924838425, 6.2643124887526144e-58)
(0.54398827848582498, 1.6718124183066529e-27)
(0.61376396774888775, 2.9634728427112909e-42)
(0.26420506962114376, 3.8757192779436134e-08)
(0.63419542726906764, 2.6858246707796961e-59)
(0.49769134075499566, 1.2832537954501601e-129)


2064

In [806]:
cp_b = np.asarray(all_features[('uniform', 'CP_B', 'BASU010101', 7)])
cp_a = np.asarray(all_features[('uniform', 'CP_A', 'BASU010101', 7)])
ei = np.asarray(all_features[('EI', 'SKOJ970101')])
ddg = skempi_df.DDG

In [807]:
c1 = pearsonr(ei, ddg)[0]
c2 = pearsonr(cp_a, ddg)[0]
c3 = pearsonr(cp_b, ddg)[0]
s = c1 + c2 + c3
a1 = c1/s
a2 = c2/s
a3 = c3/s
c1, c2, c3

(0.19216942446949448, 0.21351384288462372, 0.36314745255523262)

In [1]:
ddg_hat =  np.multiply(a1, ei) + np.multiply(a2, cp_a) + np.multiply(a3, cp_b)
pearsonr(ddg_hat, ddg)

NameError: name 'np' is not defined